In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os
import requests
import pandas

from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import HTML

from example_helpers import show_predictions, no_wrap_div, get_dataset

MDSTUDIO_URL = 'http://mdstudio-smartcyp.dev.openrisknet.org/'
#MDSTUDIO_URL = 'http://localhost:8081'
FAME_URL = 'http://fame3.dev.openrisknet.org/'

data = get_dataset()

In [ ]:
for case_name, case_data in data.items():
    
    # Get SMARTCyp prediction
    response = requests.post('{0}/som_prediction'.format(MDSTUDIO_URL),
                             files={'ligand_file': case_data['file']}, data={'cyp': case_data['cyp']}) 
    response_df = pandas.DataFrame.from_dict(response.json(), orient='index')
    response_df_prob = response_df[['Docking', 'SMARTCyp']].fillna(0).round(3)
    
    # Draw molecule
    rdmol = Chem.rdmolfiles.MolFromMol2File('{0}.mol2'.format(case_name))
    dock_svg = show_predictions(rdmol, response_df_prob['Docking'])
    smart_svg = show_predictions(rdmol, response_df_prob['SMARTCyp'])
    
    display(HTML('<h3 align="center">{0}</h3>'.format(case_name)))
    display(HTML(no_wrap_div.format(dock_svg, smart_svg)))
    display(response_df_prob.T)

In [ ]:
for case_name, case_data in data.items():
    
    response = requests.get('{0}/predict'.format(FAME_URL), params={'smiles': case['smiles']},
                             headers={"accept" : "application/json"})
    response_json = response.json()
    print(response_json['predictions'][0]['atomPredictions'])